In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
import numpy as np
from scipy.io import wavfile, loadmat
import os
import sys
print(sys.path)

['', '/Users/robinson/Dropbox/anasynth/_code', '/Users/robinson/Dropbox/anasynth/_code/as_pysrc', '/Users/robinson/Dropbox/anasynth/_code/tf_seq2seq', '/u/formes/share/packages/anaconda3/envs/DeepLearning/lib/python36.zip', '/u/formes/share/packages/anaconda3/envs/DeepLearning/lib/python3.6', '/u/formes/share/packages/anaconda3/envs/DeepLearning/lib/python3.6/lib-dynload', '/Users/robinson/Dropbox/anasynth/_code/anaconda/lib/python3.6/site-packages', '/Users/robinson/Dropbox/anasynth/_code/seq2seq', '/Users/robinson/Dropbox/anasynth/_code/anaconda/lib/python3.6/site-packages/as_pysrc_utils_poly-0.2-py3.6-macosx-10.7-x86_64.egg', '/Users/robinson/Dropbox/anasynth/_code/anaconda/lib/python3.6/site-packages/as_pysrc_utils_levinson-0.1-py3.6-macosx-10.7-x86_64.egg', '/Users/robinson/Dropbox/anasynth/_code/anaconda/lib/python3.6/site-packages/utils_bspline-1.0.0-py3.6-macosx-10.7-x86_64.egg', '/Users/robinson/Dropbox/anasynth/_code/anaconda/lib/python3.6/site-packages/opt_congrad_sc_c-1.2.2

In [2]:
'''
for each phrase, get source and predicted f0 for all syllables and assemble into one sequence, along with unvoiced # 
parts, and output into separate text files for us in the following code block 
'''

# path to source F0 CSV input files directory
csv_input_directory_path = '/Users/robinson/Dropbox/anasynth/_data/emoVC/Olivia2006/f0_raw_syllable'
# Olivia2006.e01.p01.i00.s01_s e.csv
csv_input_file_extension = '.csv'

# path to predicted F0 CSV input files directory
syll_input_directory_path = '/Users/robinson/Downloads/data/pred/20180724_160931/syllables'
# Olivia2006.e01.p01.i00.s01_s e1.csv
syll_input_file_extension = '.csv'

# path to MAT input files directory
mat_input_directory_path = '/Users/robinson/Dropbox/anasynth/_data/emoVC/Olivia2006'
# Olivia2006.e01.p01.i00.mat
mat_input_file_extension = '.mat'

# path to WAV input files directory
raw_input_directory_path = '/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/'
# Olivia2006.e01.p01.i00.1.wav
raw_input_file_extension = '.wav'

# define input file root - common to all filetypes
input_file_root = 'Olivia2006'

# define output directory and files
output_directory = os.path.join(raw_input_directory_path, 'f0compare')
if not os.path.exists(output_directory):
    os.mkdir(output_directory)
    
# open test_log.txt and split into list of lines
f_test_log = open(os.path.join(csv_input_directory_path, 'out', 'test_log.txt'), 'r')
# f_test_log = open(os.path.join('/Users/robinson/Downloads/data/pred/20180628_151243', 'test_log.txt'), 'r')  # debug
test_log = f_test_log.read().split('\n')

# define phrase range
phrase_from = 1
phrase_to = 10  #10
# define source and target emotion ranges
source_emotion_from = 1
source_emotion_to = 8  #8
# define source and target intensity ranges
source_intensity_from = 0
source_intensity_to = 0

# set sample rate 
step_s = 0.005 # 5ms, which I assume is what was used to sample the file

# file counter 
i = 0

# Turn interactive plotting off
plt.ioff()

# for each wavfile that we want to treat..
# for each phrase
for p in range(phrase_from, phrase_to + 1):
    # for each source emotion
    for e_s in range(source_emotion_from, source_emotion_to + 1):
        # for each source intensity
        for i_s in range(source_intensity_from, source_intensity_to + 1):
            # build the source file path
            filename_base = ''.join([input_file_root, 
                                        '.e', format(e_s, '02d'),
                                        '.p', format(p, '02d'),
                                        '.i', format(i_s, '02d')])
            
            # open the mat file 
            mat_filename = ''.join([filename_base, mat_input_file_extension])
            mat_filepath = os.path.join(mat_input_directory_path, mat_filename)
            mat_dict = loadmat(mat_filepath)
            
            # get list of syllables
            syll_label = mat_dict['syll_label']
            syll_label = syll_label.reshape((syll_label.shape[1],))
            
            # get syll_time from dict and transpose to column
            syll_time = mat_dict['syll_time'].T
            # print('syll_time ', syll_time)
            
            # make boolean list of voiced/unvoiced (WRONG - not just silence is unvoice, also parts of syll are)
            voiced = [True if '#' not in syll[0] else False for _, syll in enumerate(syll_label)]
            # print('voiced ', voiced)
            
            # create list to hold sequence of voiced and unvoiced f0 contour values
            # all_contours = []
            
            # create new figure for this phrase
            fig = plt.figure(figsize=(16, 8))
            plt.title(filename_base)
            plt.xlabel('Time (s)')
            plt.ylabel('Freq (Hz)')
            # plt.xlim(0, 800)
            plt.ylim(-20, 700)   
            
            values = np.arange(syll_label.shape[0]).tolist()
            # print(values)
            # print(syll_label.shape[0])
            jet = plt.get_cmap('Dark2')
            cNorm = colors.Normalize(vmin=0, vmax=values[-1])
            scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)            
            
            start_source = 0
            start_target = 0
            
            colour_counter = 0
            # voiced_syll_counter = 0
            
            # for each syllable in the wav file
            for all_syll_counter, syll in enumerate(syll_label):
                # print('******** SYL ********')
                # build the filename of the syllable
                filename_syll = ''.join([filename_base, '.s', format(all_syll_counter, '02d'), '_', syll[0],
                                         str(all_syll_counter), csv_input_file_extension])
                # print(filename_syll)
                
                # if syllable is unvoiced, or if syll file is not in test_log.txt (i.e. it's missing or empty), 
                # get the length of the unvoiced syllable and add zeroes to all_contours list
                if not voiced[all_syll_counter] or filename_syll not in test_log:
                    # print('!!! unvoiced')
                    time = syll_time[all_syll_counter, 1] - syll_time[all_syll_counter, 0]
                    # print('time ', time)
                    num_zeroes = int(time // step_s)
                    # print('num_zeroes ', num_zeroes)
                    contour_source = contour_target = [0 for _ in range(num_zeroes)]
                # if syllable is voiced, add its contents to the all_contours list
                else:
                    # build the source file path
                    source_file_path = os.path.join(csv_input_directory_path, filename_syll)
                    # load the source file and extract vars
                    source_f0_raw = np.loadtxt(source_file_path, dtype='int')
                    # iterate over the numpy array, adding items to the list
                    contour_source = [source_f0_raw[x] for x in range(source_f0_raw.shape[0])]
                    
                    # build the target file path
                    target_file_path = os.path.join(syll_input_directory_path, filename_syll)
                    # load the target file and extract vars
                    target_f0_raw = np.loadtxt(target_file_path, dtype='int')
                    # iterate over the numpy array, adding items to the list
                    contour_target = [target_f0_raw[x] for x in range(target_f0_raw.shape[0])]
                    # # increment counter
                    # voiced_syll_counter += 1
                    
                colorVal = scalarMap.to_rgba(values[all_syll_counter])
                
                plt.plot(range(start_source, start_source + len(contour_source)), contour_source, color=colorVal, 
                         alpha=0.7, linewidth=3, label='source')
                start_source += len(contour_source)
                
                plt.plot(range(start_target, start_target + len(contour_target)), contour_target, color=colorVal, 
                         alpha=0.7, linestyle=':', linewidth=3, label='predicted')
                start_target += len(contour_target)    
            
            # add legend to each figure
            plt.legend()
            plt.tight_layout()
            fig.savefig(os.path.join(output_directory, filename_base + '.png'))
            # close the fig so it never gets displayed - used with plt.ioff()
            plt.close(fig)
            # increment counters
            i += 1
            
print('done')

ValueError: invalid literal for int() with base 10: 'a250'